In [ ]:
def read_data(path):
    cities = []
    with open(path, "r") as f:
        csv.reader(f, delimiter=" ")
        for line in f:
            data = line.split(" ")
            data = [float(x.replace("\n", "")) for x in data]
            cities.append([data[0],data[1]])
    return cities

In [ ]:
def init():
    line.set_data([], [])
    return line,

def animate(i, every_sol,ax,tours,m):
    
    t = (i*tours)//(tours*m-1) + 1
    
    ant = i%m + 1
    
    title = "Tour: " + str(t) + "     Ant: " + str(ant) 
    ax.set_title(title, va='top')
    sol = every_sol[i]
    x = []
    y = []
    
    for edge in sol:
        x.append(cities[edge.ori][0])
        y.append(cities[edge.ori][1])
    
    x.append(cities[0][0])
    y.append(cities[0][1])
    
    line.set_data(x, y)
    return line,

In [ ]:
def save_animation(tours, m, every_sol, minx, miny, maxx, maxy):
    Writer = writers['ffmpeg']
    writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=100)

    global line

    fig = plt.figure(figsize=(10,6))
    ax = plt.axes(xlim=(minx-1, maxx+1), ylim=(miny-1, maxy+1))
    
    
    line, = ax.plot([],[], color='green', linewidth = 3, 
         marker='o', markerfacecolor='blue', markersize=12, label='ey')


    anim = FuncAnimation(fig, func=animate, blit=True, init_func=init, frames=len(every_sol), repeat=False, fargs=(every_sol,ax,tours,m),interval=100)
    
    name = str(tours) + "tours-" + str(m) +"ants"
    anim.save(name + '.mp4', writer=writer)

# <center> ACO para TSP (AS) </center>

In [ ]:
import os
import csv
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt 
import matplotlib
from matplotlib.animation import FuncAnimation
from matplotlib.animation import writers

Clase arista (ori,dest) donde ori es el nodo origen y dest el nodo destino

In [ ]:
class Edge:
    def __init__ (self, o, d):
        self.ori = o
        self.dest = d

La selección de la arista *edge* es posible si su nodo destino *edge.dest* no ha sido ya introducido en alguna arista contenida en *solution construction*

In [ ]:
def isFeasible(edge, solution_construction): #an edge is feasible if the ant has not chosen the destination node yet
    for e in solution_construction:
        if e.ori == edge.dest or e.dest == edge.dest:
            return False
    return True

La condición de terminación de la creación de una solución por una hormiga es True cuando la solución creada es del mismo tamaño que el número de ciudades (la hormiga pasa por todas las ciudades)

In [ ]:
def construct_termination(solution_construction, n):
    return len(solution_construction) == n - 1

La distancia entre dos ciudades se calcula como la distancia entre dos puntos (x1,y2) e (x2,y2)
<img src="img/distancia.png" alt="Drawing" style="width: 400px;"/>

In [ ]:
def distance(city1, city2):
    x1 = city1[0]
    x2 = city1[1]
    y1 = city2[0]
    y2 = city2[1]
    return pow(pow(x2-x1, 2)+pow(y2-y1,2),1/2)

La probabilidad se calcula utilizando el sendero de feromonas y la heurística de la distiancia, a través de la fórmula:
<img src="img/p_aco.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
def calculate_probability(cities, edge, pheromone_trail, solution_construction):
    alpha = 1 # >= 0
    beta = 1 # >= 1
    eta = 1 / distance(cities[edge.ori], cities[edge.dest])           # Heurística
    
    numerator = pow(pheromone_trail[edge.ori][edge.dest], alpha) * pow(eta, beta) 
    
    
    # denominador = umatorio del valor para cada arista (i, l) factible, siendo l todas los nodos factibles con origen en i
    denominator = 0 
    for l, city in enumerate(cities): 
        if edge.ori != l:
            if isFeasible(Edge(edge.ori, l), solution_construction):
                denominator += pow(pheromone_trail[edge.ori][l], alpha) * pow(1/distance(cities[edge.ori], city),beta)
    
    return numerator / denominator

Una hormiga construye su solución *solution_construction*, que empieza siendo vacía.

Partiendo del nodo origen *i = 0*, se recorren todas las ciudades obteniendo la probabilidad de utilizar cada siguiente nodo *j*. Cuando se termina con todas las ciudades factibles desde *i*, se elige una de las aristas aleatoriamente (utilizando la probabilidad).

El proceso anterior se repite, partiendo del nodo destino de la nueva arista añadida, hasta que todos los nodos (ciudades) han sido visitados.

In [3]:
def constructAntSolution(cities, pheromone_trail):
    i = 0               # El primer nodo origen es el 0
    n = len(cities)
    solution_construction = []
    while not construct_termination(solution_construction, n):
        p = []
        a = []
        for j, city in enumerate(cities):    # Se recorren todas las ciudades
            if i != j:
                edge = Edge(i, j)
                if isFeasible(edge, solution_construction):         # Si no ha sido ya visitada
                    p.append(calculate_probability(cities, edge, pheromone_trail, solution_construction))
                else:
                    p.append(0)
                a.append(j)
        
        j = np.random.choice(a, 1, p=p)[0]
        
        solution_construction.append(Edge(i, j))           # Se añade la nueva arista a solution_construction
        i = j
        
    solution_construction.append(Edge(i, 0))               # Se añade la arista final de retorno al nodo 0 para cerrar el ciclo
    return solution_construction

Se actualiza el sendero de feromonas *pheromone_trail* utilizando la siguiente fórnula:
<img src="img/ph_aco1.png" alt="Drawing" style="width: 500px;"/>
donde:
<img src="img/ph_aco2.png" alt="Drawing" style="width: 500px;"/>

In [4]:
def updatePheromone(pheromone_trail, solutions, cities):
    evaporation_rate = 0.5 #ⲣ = evaporación de la feromona
    Q = 500 #constante
    
    # Evaporation
    for i in range(len(pheromone_trail)):
        for j in range(len(pheromone_trail)):
            pheromone_trail[i][j] *= (1 - evaporation_rate)
    
    for ant_solution in solutions:
        for edge in ant_solution:
            Lk = distance(cities[edge.ori], cities[edge.dest])     # Heurística
            ph = Q / Lk                                            # Cantidad de feromona depositada Δt
            pheromone_trail[edge.ori][edge.dest] += ph

Algoritmo general del ACO:
<img src="img/aco.png" alt="Drawing" style="width: 300px;"/>

In [ ]:
def aco(cities, m, n, tours):  
    pheromone_trail = []
    
    # 1.   Ajustado en el método que llama a este método (m, n, tours)
    
    '''every_sol = []    # Para la animación '''

    # 2.   Inicializar sendero de feromonas, todo a 0
    for i in range(n):
        pheromone_trail.append([0.000000001]*n)
    
    for t in range(tours):
        solutions = []
       
        # Cada hormiga genera una solución, que se añade a la lista de soluciones del tour actual *solutions*
        for ant in range(m):
            
            # 3.   Construcción de la solución por la hormiga ant
            sol = constructAntSolution(cities, pheromone_trail)

            solutions.append(sol)
            
            ''' every_sol.append(sol)         # Para la animación '''


        # 4.   Actualización del sendero de feromonas utilizando las soluciones que han generado las m hormigas
        updatePheromone(pheromone_trail, solutions, cities)
    
    ''' Para la animación
    x, y = [], []
    for c in cities:
        x.append(c[0])
        y.append(c[1])
    minx = min(x)
    maxx = max(x)
    miny = min(y)
    maxy = max(y)'''
    
    
    '''save_animation(tours, m, every_sol, minx, miny, maxx, maxy)'''
    
    '''return every_sol        # Para la animación'''

In [ ]:
path = os.path.join(os.getcwd(), 'dataset', 'tsp', "instance7.txt")
cities = read_data(path)

m = 15 # Número de hormigas
n = len(cities) # Número de ciudades

tours = 50 # Número de tours
iterations = 1


for it in range(iterations):
    
    ants.append(m)
    
    #start=datetime.now()
    
    '''every_sol = aco(cities, m, n, tours)    # Para la animación'''
    
    aco(cities, m, n, tours)
    
    #time = datetime.now()-start

    m+=5